##  감성체인 생성

In [1]:
import os
import time
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain.chains import LLMChain  # LLMChain을 사용하여 Chain을 구성합니다
import logging

In [2]:
# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

In [3]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# OpenAI LLM 모델 설정
llm = ChatOpenAI(openai_api_key=api_key, model_name="gpt-4-turbo-preview", temperature=0.3)

# 감성 분석 기본 프롬프트
prompt_template = PromptTemplate.from_template(
    """你是一名研究中国的社会科学家，你的任务是分析中国新闻对“美日韩”三边关系的报道，并从中国的视角分析这些报道的情感**。

      **评分标准**：
      - 强烈负面 (-5): 极端消极
      - 负面 (-3 ~ -4): 较为消极
      - 轻微负面 (-1 ~ -2): 略微消极
      - 中立 (0): 纯粹的事实陈述，不带有任何情感倾向。
      - 轻微正面 (+1 ~ +2): 略微积极
      - 正面 (+3 ~ +4): 较为积极
      - 强烈正面 (+5): 极端积极

      **请严格分析以下新闻文章的所有句子，并为每个句子提供一个情感分数。所有提供的句子都必须进行情感分析，不得遗漏任何句子**。
      **新闻文章内容:**
      {sentences}

      **请返回每个句子的情感分数，每行一个分数，不要包含任何额外的文本、注释或解释。**
    """
)

In [ ]:
# prompt_template = PromptTemplate.from_template(
#     """你是一名研究中国的社会科学家，你的任务是分析中国新闻对“美日韩”三边关系的报道，并从中国视角进行情感分析。请特别关注涉及中国核心利益（主权、安全、外交）的内容，并确保评分符合中国立场。**#
#
#       **评分标准**：
#       - 强烈负面 (-5): 极端消极
#       - 负面 (-3 ~ -4): 较为消极
#       - 轻微负面 (-1 ~ -2): 略微消极
#       - 中立 (0): 纯粹的事实陈述，不带有任何情感倾向。
#       - 轻微正面 (+1 ~ +2): 略微积极
#       - 正面 (+3 ~ +4): 较为积极
#       - 强烈正面 (+5): 极端积极
#
#       **分析要求**
#       - 严格分析所有句子，不得遗漏。
#       - 句子涉及中国核心利益时，优先考虑中国立场。
#       - 句子包含多个情感因素时，按对中国的总体影响评分。
#
#       **新闻文章内容:**
#       {sentences}
#
#       **请返回每个句子的情感分数，每行一个分数，不要包含额外文本或解释。**
#     """
# )

In [4]:
# 체인 구성
sentiment_chain = LLMChain(prompt=prompt_template, llm=llm)

C:\Users\82104\AppData\Local\Temp\ipykernel_20936\1462471065.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  sentiment_chain = LLMChain(prompt=prompt_template, llm=llm)


## 데이터 로드

In [12]:
import json

# JSON 파일 로드
file_path = "../../data/0310/10/split_by_year/2020.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"총 {len(data)}개의 기사 로드 완료")

총 38개의 기사 로드 완료


## 감성분석 수행 함수

In [13]:
import time
import random
import json

# 감성 분석 수행 함수
def analyze_sentiment(article):
    """기사 전체 본문을 입력하고 문장별 감성 점수를 분석하는 함수"""
    try:
        # API 요청 전에 딜레이 추가 (랜덤 딜레이)
        delay_time = random.uniform(10, 15)  # 5초에서 10초 사이의 랜덤 딜레이
        time.sleep(delay_time)

        # 기사 본문만 전달
        input_data = {"sentences": article["sentences"]}

        # `sentiment_chain.run()`을 사용하여 데이터 처리
        response = sentiment_chain.invoke(input_data)  # input_data 전달

        print(f"Response format: {response}")

        # 'text' 키에서 감성 분석 결과를 추출하여 'sentiment_responses'에 저장
        if isinstance(response, dict):
            if 'text' in response:
                article["sentiment_responses"] = response['text'].strip()  # 감성 분석 결과 저장
            else:
                article["sentiment_responses"] = "Error: 'text' key not found in response"
        else:
            article["sentiment_responses"] = "Error: Response is not a dictionary"

        return article
    except Exception as e:
        article["sentiment_responses"] = f"Error: {str(e)}"  # 오류 발생 시 오류 메시지 저장
        return article  # 오류 발생 시에도 응답 내용 저장

def process_article(article):
    """GPT 응답 후처리"""
    sentiment_response = article.get("sentiment_responses", "")

    # 문장이 없는 경우 바로 반환
    if not sentiment_response:
        return article

    sentiment_scores = []
    for line in sentiment_response.split("\n"):
        try:
            # 문장별 감성 점수를 처리
            sentiment_scores.append(float(line.strip()))  # 각 점수 추가
        except (ValueError, IndexError):
            continue
    article["sentiment_scores"] = sentiment_scores
    article["avg_sentiment_score"] = round(sum(sentiment_scores) / len(sentiment_scores), 2) if sentiment_scores else None
    return article

# 백오프 전략을 위한 재시도 함수
def retry_request(article, max_retries=5, delay_factor=2):
    """백오프 전략을 적용하여 감성 분석 요청을 재시도하는 함수"""
    for attempt in range(max_retries):
        result = analyze_sentiment(article)  # 감성 분석 호출
        if "Error" not in result["sentiment_responses"]:  # 에러가 없으면 바로 반환
            return result
        else:
            print(f"Retry {attempt + 1}/{max_retries} for {article['title']} due to error.")
            time.sleep(random.uniform(30, 40) * delay_factor)  # 재시도 전 대기 (시간 늘리기)
            delay_factor *= 2  # 다음 재시도 때는 더 긴 시간 대기
    return result  # 재시도 후에도 실패하면 마지막 결과 반환

def run_sentiment_analysis(data, output_file):
    """JSON 파일을 로드하여 감성 분석을 순차적으로 실행하고 결과를 저장하는 함수"""
    results = []

    # 기사별 순차 실행
    for article in data:
        processed_article = retry_request(article)  # 감성 분석 수행
        processed_article = process_article(processed_article)  # 후처리

        if processed_article is not None:
            results.append(processed_article)  # 결과 저장

        # 중간 저장 (기사별로 즉시 JSON 파일 업데이트)
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(results, f, ensure_ascii=False, indent=4)
        print(f" '{article['title']}' 분석 완료.")

    print(f" 전체 기사 감성 분석 완료. {output_file}")


In [15]:
output_path = "../../data/0310/results/2020.json"
run_sentiment_analysis(data, output_path)

2025-03-17 17:29:05,372 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'sentences': ['在朝鲜给美国调整对朝政策的年底期限到期两周之后美国选择同其在亚洲的两国重要盟友日本韩国先行展开协调沟通。', '据美国国务院网站1月14日消息美国国务卿蓬佩奥当日在美国加州同来访的日本外相茂木敏充及韩国外长康京和举行三方会谈三方强调美韩同盟美日同盟以及美日韩三边合作的重要性。', '此外三国之间也举行了双边会谈。', '虽然三方都强调了彼此的合作与共识但是从种种信息中仍然可见三方对于朝鲜半岛和中东局势的不一致态度。', '半岛局势韩国与美日立场不一致 美日韩三国尤其是美国作为朝鲜半岛局势重要的当事国和相关方迫切希望通过这次会议释放出统一的对外信号。', '然而最终从各方的表述来看三方在具体问题尤其是对朝制裁与合作开展的问题上并不完全齐心。', '美国和日本趋向于继续对朝强硬的立场。', '据日本英文媒体日本时报Japan Times14日报道美日韩三方在会谈期间敦促朝鲜方面不要采取任何军事挑衅行为并表示将继续寻求开展无核化谈判但同时也强调现阶段不会倾向于取消对朝经济制裁。', '日本时报称茂木敏充在14日的联合记者会上重复了相关内容但韩方似乎并没有表现出完全一致。', '就在此前一天韩国总统文在寅在新年演讲中刚刚表态自己将寻求联合国逐步豁免对朝鲜的制裁以改善朝韩关系。'], 'text': '0\n0\n0\n-1\n-2\n-2\n-1\n-1\n-1\n-1'}
 '美日韩外长会举行,半岛问题、中东局势方面三国并不齐心-1' 분석 완료.


2025-03-17 17:29:20,116 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'sentences': ['文在寅说制裁本身制裁限制了韩国和朝鲜的合作。', '据韩联社15日报道康京和在韩美外长会中介绍了文在寅的半岛合作构想并重申坚持推进韩朝合作的决心。', '康京和还表示朝美对话和韩朝对话应互补且形成良性循环而在特定时间点有可能朝美对话在先也有可能韩朝对话在先。', '韩联社引述韩国统一部副发言人金银汉消息指出在朝韩下一阶段的合作项目中旅游等部分项目只需经韩朝双方协商即可推进。', '值得注意的是在美日韩三国外长举行会谈的同时日本防卫大臣河野太郎目前也在美国访问于14日与美国国防部长埃斯珀和美国总统国家安全事务助理奥布莱恩举行了会谈。', '据日本共同社的消息埃斯珀在与河野的会晤中强调美日韩三方军事演习以及情报分享仍将持续。', '中东护航联盟韩国未明确回应 除了朝鲜半岛问题之外近来骤然发酵的美伊局势也成为此次美日韩外长会的讨论重点。', '美国也希望借此机会敦促韩国也向中东派兵不过韩方并没有给予明确回应。', '据韩联社14日报道蓬佩奥在韩美外长会谈上提出所有国家都需共同努力为稳定霍尔木兹海峡及中东局势作贡献。', '有分析认为此举实际上是美方就海湾地区的护航联盟问题向韩国施压。'], 'text': '-1\n0\n0\n0\n-1\n0\n-1\n-2\n-1\n-2'}
 '美日韩外长会举行,半岛问题、中东局势方面三国并不齐心-2' 분석 완료.


2025-03-17 17:29:35,723 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

Response format: {'sentences': ['日本内阁官房长官加藤胜信12日在谈及拜登有关尖阁诸岛的表态时称这说明在印太地区日益严峻的安保环境下下届美国政府也会继续加强日美同盟的遏制能力。', '日本媒体对菅义伟与拜登的首次通话给予高度关注。', '东京新闻称菅义伟在通话中直接称呼拜登为美国下任总统说明他或许有意建立与拜登的私人关系。', '不过日本时事通讯社认为与G7成员中的英法德领导人相比日本政府现在才通话反应太慢了。', '针对拜登有关尖阁诸岛的表态日本经济新闻认为这反映出日本与美国新政府在对华政策的合作方面将首先从确认安保大原则开始。', '拜登尚处于赢得大选阶段就如此明确表态可以说较为罕见。', '这向国际社会传递了即使美国政府更迭也不会降低对华施压的信号。', '一名了解日本时政的分析人士12日对环球时报记者表示拜登在尚未正式被认定当选总统的情况下急于表态日美安全保障条约第5条适用钓鱼岛是在安抚菅义伟政府对日美关系可能发生变数的担忧情绪。', '美国当前正处于大选后结果尚未确定的混乱状态拜登作为老成持重的建制派政治精英深知第一时间就日美安全保障条约适用对象问题作出承诺无异于给菅政府以定心丸并为今后在亚太政策上获得日本的全力配合提供信任担保。', '但拜登为日本背书的错误言论将给东亚地区的稳定带来消极影响。'], 'text': '0\n+1\n+1\n-1\n+1\n+2\n+2\n+2\n+3\n-3'}
 '急于巩固同盟关系？日韩领导人同日与拜登通话-2' 분석 완료.


2025-03-17 17:52:40,706 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'sentences': ['中国外交部发言人汪文斌12日就有关问询回应说钓鱼岛及其附属岛屿是中国固有领土。', '美日安保条约是冷战产物不应损及第三方利益更不应危害地区和平稳定。', '在菅义伟之后韩国总统文在寅12日也与拜登进行了约14分钟的通话。', '双方谈及韩美同盟朝核新冠肺炎疫情等问题。', '文在寅强调韩美同盟在过去70年已成为半岛维持和平与繁荣的基石韩方将同美方紧密沟通以发展面向未来的韩美同盟关系实现半岛无核化及永久和平。', '拜登表示将与韩方紧密合作解决朝核问题。', '他还强调韩国是印太地区安全与繁荣的关键美国将继续履行对韩国做出的防卫承诺并与韩国紧密合作解决朝核问题。', '韩联社就此表态分析认为除想强调韩美同盟关系外拜登其实是在间接邀请韩国加入由美国主导的旨在牵制中国的印太战略。', '韩国中央日报称美国最早用关键来形容韩国和韩美同盟是在奥巴马执政时期。', '特朗普执政期间较少使用该词。'], 'text': '0\n-2\n0\n0\n+2\n+1\n+1\n-1\n0\n0'}
 '急于巩固同盟关系？日韩领导人同日与拜登通话-3' 분석 완료.


2025-03-17 17:52:55,464 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'sentences': ['美国此前一般用关键来描述美日同盟关系奥巴马对韩国使用该词被解读为韩国在印太地区的战略价值和重要性提高。', '韩媒认为拜登此次重新用关键一词来形容韩国对韩国来说是个负担。', '韩国News1新闻网还提到拜登十分重视美日韩三国合作但他与文在寅通话时并未谈到韩日关系。', '值得注意的是与日媒一样韩媒也觉得本国总统与拜登通话慢了晚日本一步。', '对此青瓦台一名核心人士解释称谁先通话不重要。'], 'text': '0\n-2\n0\n-1\n0'}
 '急于巩固同盟关系？日韩领导人同日与拜登通话-4' 분석 완료.
 전체 기사 감성 분석 완료. ../../data/0310/results/2020.json
